# A bioinformatics exercise
This notebook uses a small bioinformatics exercise to show aspects of the Python programming 
language in the context of a real(ish) data processing activity.

We will be reading, writing, and manipulating text files and running a small sequence alignment
program.  Over the course of this we will cover programming topics such as:

   * Built-in Python types including strings, ints, floats
   * Python code blocks including if/then/else, for loops, functions,
     and context managers
   * Data structures like lists and dictionaries
   * System calls, including multiprocessing Pools
   
Additional topics including Python packages and environments and the object-orientation of Python
will be covered elsewhere.

In this course we will often demonstrate something and then follow that with a quick exercise where you try to apply it to the bioinformatics exercise.  Then we'll go over what it and take any questions. 

# 1. Navigating [JupyterLab](https://jupyterlab.readthedocs.io/)
    
  * you've already discovered the lefthand file navigator, double click to open files or enter a folder
  * you will notice some blocks are text and some are code boxes
  * double click on a text block and you will see the mark down version
  * notice there is a play button at the top of the file
  * inside the text block you clicked on push the play button to see the formatted text
  * code blocks are interpreted by python click in the code block below and type 1, then push the play button
  * you will see the output of the code below the code block (note that you cannot click on this)
  * jupyter by default gives any output of the last line in the code block
  * jupyter also displays any output from functions in the code block


In [29]:
1

1

In [30]:
1
2
3

3

In [31]:
print(1)
print(2)
3

1
2


3

# 2. Numbers and strings at the interactive interpreter

Math works as expected 

In [32]:
1+1

2

Strings are output by the interpreter with single quotes

In [33]:
'hello world'

'hello world'

# 3. Hello world with the print function

Let's try one more hello world, type "print('hello world')"

In [34]:
print('hello world')

hello world


print is a builtin python language function and 'hello world' is a parameter passed to the function.

python language is open source:

https://github.com/python/cpython/blob/master/Python/bltinmodule.c#L1821

More about functions shortly.

# 4. Python types and variables

Rather than working with raw numbers and strings at the interpreter it can be handy to assign them to variables that can be used multiple times

Let's create a variable called string (the name is a spoiler alert for the type, ha). In Python, the equal sign means "assignment".  Double equal ("==") tests equality.

In [35]:
string = 'hello world'

In [36]:
print(string)

hello world


Use the function type() to see what python type 'hello world' is

In [37]:
type('hello world')

str

All variables are objects in python, we'll examine more of what that means later, but str is a type of object.   Our variable string points to an object of the type str.

In [38]:
type(string)

str

### Exercise 4.1: Let's try some of Python's basic types

Set a variable called number to 1, then check the type

Set a variable called number to '1' and see what the type is then

Set a variable to the number 1.5 and see what type it is

Set a variable to True and see what type it is

In preparation for reading an annotations file set a variable named file_name to the file path 'data/chr12/annotations.1.txt

# 5. String concatenation

Strings can be concatenated with the '+' operator.  Non-strings must be
converted first with _str()_

In [44]:
'python ' + 'is ' + 'number ' + str(1)

'python is number 1'

In [45]:
data_dir = 'data'
project = 'chr12'
name = 'annotations'
version = 1
ext = 'txt'

### Exercise 5.1: use concatenation of the above variables to create a variable called file_name which is the same as the one from end end of Exercise 1  (file_name = 'data/chr12/annotations.1.txt')

# 6. Functions

A function is a block of code that can be run on 0 or more arguments using the "call" operator _()_ and may return some value. 

In [47]:
def hello_world():
    print('hello world')
    
hello_world()

hello world


In [48]:
def python_is(descriptor, action):
    string = 'Python is ' + str(descriptor) + ' everyone should ' + action + ' it '
    return string

python_is('fun', 'try')

'Python is fun everyone should try it '

You can add a multiline comment, surrounded by ''', to a function for documentation.  Functions are objects too and you can see this comment by passing the object to the help function. 

In [49]:
def python_is(descriptor, action):
    '''
    Concatenates the string 'Python is ' with descriptor and ' everyone should ' with action 
    '''
    string = 'Python is ' + str(descriptor) + ' everyone should ' + action + ' it '
    return string

python_is('easy', 'learn')

'Python is easy everyone should learn it '

In [50]:
help(python_is)

Help on function python_is in module __main__:

python_is(descriptor, action)
    Concatenates the string 'Python is ' with descriptor and ' everyone should ' with action



Positional arguments must be passed to the function in the order they are listed

In [51]:
def python_is(descriptor, action):
    '''
    Concatenates the string 'Python is ' with descriptor and ' everyone should ' with action 
    '''
    string = 'Python is ' + str(descriptor) + ' everyone should ' + action + ' it '
    return string

python_is('learn', 'easy')

'Python is learn everyone should easy it '

### Exercise 6.1: Make a function called get_annotation_file_name which takes the 5 variables we used in exercises 2 as parameters and returns the concatenated file path (hint you can copy paste that part from exercise 2)

In [3]:
def get_annotation_file_name(
    data_dir, 
    project, 
    version, 
    name, 
    ext):

    '''
    Concatenates data_dir and project_name for path.  "annotations.<version>.<extension>" is the file name.
    '''



use the below to call your function and test it

In [ ]:
get_annotation_file_name(data_dir, project, version, name, ext)

# 7. Function arguments

You can specify defaults when it makes sense, but positional arguments must come first

In [54]:
def python_is(descriptor, action = 'learn'):
    '''
    Concatenates the string 'Python is ' with descriptor and ' everyone should ' with action 
    '''
    string = 'Python is ' + str(descriptor) + ' everyone should ' + action + ' it '
    return string

python_is('easy')

'Python is easy everyone should learn it '

Arguments that don't have a default must be specified

In [55]:
result = python_is()

TypeError: python_is() missing 1 required positional argument: 'descriptor'

They can also be treated as keyword arguments and specified in arbitrary order

In [57]:
result = python_is(action='enjoy', descriptor='useful')
result

'Python is useful everyone should enjoy it '

### Exercise 7: Defaults for annotation file name function

#### 7.1. Copy the function you wrote from exercise 6 and add defaults for extension and name. 

#### 7.2 Run our annotation file function with only the required arguments.

#### 7.3. Specify the name with a different value

#### 7.4. Try specifying the arguments as keyword arguments in a different order than their position in the function definition

# 8. Formatted strings

Python supports both positional and named string template substitution.  See the
[Pyformat page](https://pyformat.info/) for details

#### String concatenation is expensive because Python strings are immutable

In [62]:
file_name = get_annotation_file_name(data_dir, project, version)

In [63]:
file_name[0] = 'a'

TypeError: 'str' object does not support item assignment

#### Old style string formatting is common

In [64]:
address = '%d %s %s %s,%s' % (52, 'Oxford', 'Street', 'Cambridge', 'MA')
address

'52 Oxford Street Cambridge,MA'

#### format function is more readable and powerful

The format function of strings allows for positional substitution like old style
formatting, but also supports named place holders and rich formatting options

_format()_ is a good example of functions that are part of defined on object-oriented 
"classes" and used on instances called "objects".

You can access the properties of an object, both its public functions and public properties through the dot notation (.)

In [65]:
address = '{} {} {} {},{}'.format(52, 'Oxford', 'Street', 'Cambridge', 'MA')
address

'52 Oxford Street Cambridge,MA'

Types can be enforced using type specifiers like ':d'

In [66]:
address = '{:d} {} {} {},{}'.format(52, 'Oxford', 'Street', 'Cambridge', 'MA')
address

'52 Oxford Street Cambridge,MA'

In [67]:
address = '{:d} {} {:.2}. {},{}'.format(52, 'Oxford', 'Street', 'Cambridge', 'MA')
address

'52 Oxford St. Cambridge,MA'

Keyword arguments can be really helpful for readability

In [68]:
address = '{number:d} {street} {suffix:.2}. {city},{state}'.format(
    number=52, 
    street="Oxford", 
    suffix="Street",
    city="Cambridge", 
    state="MA" 
)
address

'52 Oxford St. Cambridge,MA'

### Exercise 8. use string formatting to rewrite the get_annotation_file_name function

#### 8.1 use old style formatting 

In [ ]:
file_name = get_annotation_file_name(data_dir, project, version)
file_name

#### 8.2 use the format function to rewrite the get_annotation_file_name function

In [ ]:
file_name = get_annotation_file_name(data_dir, project, version)
file_name

# 9. Lists 

Like arrays in other languages, Python lists are a group of items that can be indexed by an integer.

Lists are initialized with [] or list() and indexing starts with zero.

In [73]:
path_elements = ['nano-course', 'python', 'data', 'chr12']

In [74]:
path_elements[0]

'nano-course'

In [75]:
path_elements[2]

'data'

Check the length with _len()_

In [76]:
len(path_elements)

4

You can use negative indexes

In [77]:
path_elements[-1]

'chr12'

Slices can be taken from lists using [:] notation.  Don't forget that the upper bound index is not included.

In [78]:
path_elements[0:2]

['nano-course', 'python']

And you can slice with negative indexes

In [79]:
path_elements[-2:-1]

['data']

strings can be sliced just like lists 

In [80]:
hello = 'Greetings'
hello[0]

'G'

if you leave a number out with the : notation it assumes you mean until the end or from the beginning

In [81]:
hello[:5]

'Greet'

In [82]:
hello[5:]

'ings'

Lists can be appended to

In [83]:
path_elements.append('annotations.1.txt')
path_elements

['nano-course', 'python', 'data', 'chr12', 'annotations.1.txt']

and extended

In [84]:
full_path = ['Users','maria']
full_path.extend(path_elements)
print(full_path)

['Users', 'maria', 'nano-course', 'python', 'data', 'chr12', 'annotations.1.txt']


List elements are mutable

In [85]:
path_elements[1] = 'R'
path_elements

['nano-course', 'R', 'data', 'chr12', 'annotations.1.txt']

You can also create an immutable list, a tuple, using parens.  These are generally used for data that does not changes and should remain together such as a compound key like ('category', 'color') might be the key for a group of products. 

In [86]:
path_tuple = ('nano-course', 'python', 'data', 'chr12')
path_tuple[1] = 'x'

TypeError: 'tuple' object does not support item assignment

## Exercise 9. try out lists

#### 9.1 create a list called list1 with 3 objects 1, 2, 3

#### 9.2 append 4, 5, and 6 to the list

#### 9.3 create a list2 with 7, 8, 9 and then extend list1 with list2

#### 9.4 print out the element 1 of the list1 and the element 9 of list1

# 10. Iterating, joining and splitting lists

We can iterate a list with a for loop.

In [91]:
for path_element in path_elements:
    print(path_element)

nano-course
R
data
chr12
annotations.1.txt


If you need the index, _enumerate()_

In [92]:
for i, path_element in enumerate(path_elements):
    print(i, path_element)

0 nano-course
1 R
2 data
3 chr12
4 annotations.1.txt


Strings act like lists...

In [93]:
data_dir[-1]

'a'

In [94]:
for ch in data_dir:
    print(ch)

d
a
t
a


but they are not mutable

In [95]:
data_dir[1] = 'x'

TypeError: 'str' object does not support item assignment

#### You can join list elements into a string with the join function 

In [96]:
address_list = ['52', 'Oxford', 'Street', 'Cambridge', 'MA']
address_display = ' '.join(address_list)
address_display

'52 Oxford Street Cambridge MA'

#### You can also split a string into a list, space is the default separator

In [97]:
address_list2 = address_display.split()
address_list2

['52', 'Oxford', 'Street', 'Cambridge', 'MA']

## Exercise 10. More with lists 

#### 10.1 use join on path_elements to create a '/' separated path

#### 10.2 We can redefine the function get_annotation_file_name using a list which contains 3 elements (data_dir, project, and a formatted string of the name, version, ext) then join with '/'

In [ ]:
file_name = get_annotation_file_name(data_dir, project, version)
file_name

#### 10.3 create a list of your own with at-least 3 items in it

#### 10.4 iterate over the list you created using a for loop and print out each element

#### 10.5 split the following string into a list using the split() function

In [ ]:
header = 'Name;Email;Address;City;State;Country'


# 11. Modules 

A module is a file with Python definitions and statements.  The _import_ statement allows you to use those definitions in your code

The creation of modules is how Python libraries are made and shared.

For example, if you're doing several projects with DNA sequence, you might like a module that had common DNA sequence manipulations.  In a file called dna.py you could define several functions and data that you might use repeatedly:

```python
DNA_COMPLEMENT = {
    'A': 'T',
    'T': 'A',
    'C': 'G',
    'G': 'C',
}

def reverse_complement(dna):
    '''
    Return the reverse complement of the DNA sequence
    '''
    complement = []
    for base in reversed(dna):
        complement.append(DNA_COMPLEMENT[base.upper()])
    return complement


def translate(dna, frame=0):
    '''
    Translate a string of dna sequence into protein sequence using the given frame
    '''
    protein_sequence = []
    for i in range(frame, len(dna), 3):
        ...
    return ''.join(protein_sequence)

def transcribe(dna):
    '''
    Convert DNA into RNA
    '''
    return dna.replace('T', 'U')
```


To use the functions in this file, you would have to either import the entire module and use the functions (via the dot operator):

```python
import dna

transcript_sequence = 'TACGATCGATCGATCGATTATCGATCAGTCA'
protein_sequence = dna.translate(transcript_sequence)
```

Or you could import specific functions from the file

```python
from dna import translate

protein_sequence = translate('TACGATCGATCGATCGATTATCGATCAGTCA')
``` 
    
The _from_ keyword will get you to the thing you want to import, but the import is what you're allowed to use in your code

##### Python modules can be organized in directories traversed by _from_

If the _dna.py_ file described above is placed under a path, e.g. _seqlib/seq/nuc/dna.py_, functions could be accessed using the _from_ keyword with dots replacing the path separator.

```python
from seqlib.seq.nuc.dna import transcribe
```
    
This will work, but a file named \_\_init\_\_.py must be present in each of the directories

##### Python starts looking for modules based on the value of _sys.path_, which may include PYTHONPATH, the current directory, and ~/.local

    [akitzmiller@bioinf01 ~]$ echo $PYTHONPATH
    /odyssey/rc_admin/sw/admin/rcpy:

    [akitzmiller@bioinf01 ~]$ pwd
    /n/home_rc/akitzmiller

    [akitzmiller@bioinf01 ~]$ python
    Python 2.7.5 (default, Apr  9 2019, 14:30:50) 
    [GCC 4.8.5 20150623 (Red Hat 4.8.5-36)] on linux2
    Type "help", "copyright", "credits" or "license" for more information.
    
    >>> import sys, os
    
    >>> os.environ['PYTHONPATH']
    '/odyssey/rc_admin/sw/admin/rcpy:'
    
    >>> print '\n'.join(sys.path)

    /odyssey/rc_admin/sw/admin/rcpy
    /n/home_rc/akitzmiller
    /usr/lib64/python27.zip
    /usr/lib64/python2.7
    /usr/lib64/python2.7/plat-linux2
    /usr/lib64/python2.7/lib-tk
    /usr/lib64/python2.7/lib-old
    /usr/lib64/python2.7/lib-dynload
    /usr/lib64/python2.7/site-packages
    /usr/lib64/python2.7/site-packages/gtk-2.0
    /usr/lib/python2.7/site-packages
    >>> 


The _os_ module must be imported and contains functions that are sensitive to the operating system

os.path.join will join each parameter with a '/' to create a path

In [104]:
os.path.join()

NameError: name 'os' is not defined

Notice we get an error above because there is no os currently defined. 

Everything you use in a Python script must either be a built-in (e.g. print), defined in your code (e.g. , get_annotation_file_name) or imported

In [106]:
import os

##### You can find where a module comes from using the \_\_file\_\_ property of the module
Seriously, everything is an object

In [107]:
os.__file__

'/opt/conda/lib/python3.7/os.py'

In [108]:
help(os.path.join)

Help on function join in module posixpath:

join(a, *p)
    Join two or more pathname components, inserting '/' as needed.
    If any component is an absolute path, all previous path components
    will be discarded.  An empty last part will result in a path that
    ends with a separator.



In [109]:
os.path.join('nano-course', 'python', 'data', 'chr12')

'nano-course/python/data/chr12'

let's import another module like datetime

In [110]:
import datetime

In [111]:
today = datetime.datetime.now()
today

datetime.datetime(2020, 1, 14, 18, 30, 38, 431649)

since what we really want to use is the datetime function inside the datetime module we can import that directly with the from keyword

In [112]:
from datetime import datetime

In [113]:
datetime(2020, 1, 21)

datetime.datetime(2020, 1, 21, 0, 0)

## Exercise 11

#### 11.1 rewrite the get_annotation_file_name function again using the os.path.join function

In [ ]:
file_name = get_annotation_file_name(data_dir, project, version)
file_name

#### 11.2 import the random module and use the random() function to generate a random number

# 12. If else

Python's if statement allow flow control by executing blocks of code only when conditions are met.  Else if is shortened to elif, there can be zero or more elif parts, and the else part is optional.

In [118]:
x = int(input("input an int please: "))

input an int please:  4


In [119]:
if x > 0:
    print('positive')
elif x == 0:
    print('zero')
else:
    print('negative')

positive


## Exercise 12 species to common name

#### 12.1 write an input string called org for organism

#### 12.2 set a variable common_name to empty string ('') then write an if else block that maps a few species to the common name and then an else block set any other species to the common_name 'Unknown'

In [ ]:
'''
here are the species -> common names that your function should use
Homo sapiens -> Human
Pan troglodytes -> Chimp
Macaca mulatta -> Macaque
other -> Unknown
'''


#### 12.3 write a function called get_common_name() that takes an organism and returns the common name or Unknown (feel free to copy and paste some of your code from above)

In [ ]:


# you can test your function with this line
get_common_name('Homo sapiens')

# 13. Open a file 

First let's open data/samples.txt by double clicking so we can take a look at what the content looks like

In Python you interact with a file by opening a file handle in a particular mode, in this case 'read'.  A file handle is a lot like a pointer to the next part of the file that you're going to read.

In [123]:
sample_file = 'data/samples.txt'
fileh = open(sample_file, 'r')

Read it all into a single string using _read()_

In [124]:
fileh.read()

'Sample\tSpecies\t\nsample1\tHomo sapiens\t\nsample2\tPan troglodytes\t\nsample3\tHomo sapiens\t\nsample4\tMacaca mulatta\t\n'

Read it into a list of lines using _readlines()_.  You may need to re-open the file, because the fileh is now pointing to the end.

In [125]:
fileh.readlines()

[]

In [126]:
fileh = open(sample_file, 'r')

In [127]:
lines = fileh.readlines()
lines

['Sample\tSpecies\t\n',
 'sample1\tHomo sapiens\t\n',
 'sample2\tPan troglodytes\t\n',
 'sample3\tHomo sapiens\t\n',
 'sample4\tMacaca mulatta\t\n']

Or, especially if your file is large, you can read one line at a time using _for_ because a file handle acts like a list. <br/>Using print() will convert the \t and \n into tabs and newlines respectively

In [128]:
fileh = open(sample_file, 'r')

In [129]:
for line in fileh:
    print(line.strip())

Sample	Species
sample1	Homo sapiens
sample2	Pan troglodytes
sample3	Homo sapiens
sample4	Macaca mulatta


In [130]:
fileh = open(sample_file, 'r')

for line in fileh:
    if not line.startswith('Sample'):
        print(line.strip())

sample1	Homo sapiens
sample2	Pan troglodytes
sample3	Homo sapiens
sample4	Macaca mulatta


Using a context manager (_with_ _as_) is a good way to ensure that the file will close when you're done with it.

In [131]:
with open(sample_file, 'r') as fileh:
    for line in fileh:
        if not line.startswith('Sample'):
            print(line.strip())

sample1	Homo sapiens
sample2	Pan troglodytes
sample3	Homo sapiens
sample4	Macaca mulatta


We can see that the fileh is closed because we are using the context manager.

In [132]:
fileh.closed

True

## Exercise 13. Try opening the annotations.1.txt file

#### 13.1 open the data/chr12/annotations.1.txt file and get a sense of the content, how are the values delimited?

#### 13.2 open the annotations file add the lines to a list and then print them out 

#### 13.3 print out everything but the header row, optionally store the header row in a variable called header

#### 13.4 now use the split function on each line before you add it to the list (as you recall the split function splits a string into a list)

#### 13.5 using list indexing get the second element of the first row of lines

#### 13.6 loop through each line of data and use the get_common_name function you wrote to get the common name for that row and add it to a list called common_names

#### Report out the unique organism common names using a list

# 14. Sets and Dictionaries

A _set_ is a collection of unique elements that can participate in set operations like unions and intersects

In [139]:
model_organisms = set(['Human', 'Mouse', 'Fruit fly', 'Macaque', 'Zebrafish'])
model_organisms

{'Fruit fly', 'Human', 'Macaque', 'Mouse', 'Zebrafish'}

you can add elements to a set with the function add which is similar to list append

In [140]:
model_organisms.add('E. coli')
model_organisms

{'E. coli', 'Fruit fly', 'Human', 'Macaque', 'Mouse', 'Zebrafish'}

But it will not add duplicates

In [141]:
model_organisms.add('Human')
model_organisms

{'E. coli', 'Fruit fly', 'Human', 'Macaque', 'Mouse', 'Zebrafish'}

A dictionary as a set of key: value pairs.  The keys are unique (within one dictionary). A pair of braces creates an empty dictionary: {}. You can add keys with a braket notation setting them equal to values.

In [142]:
capitals = {}
capitals['MA'] = 'Boston'
capitals['NH'] = 'Concord'
capitals

{'MA': 'Boston', 'NH': 'Concord'}

A dictionary can also be initialized with values using a colon to separate keys and values.

In [143]:
capitals = {'MA': 'Boston', 'NH': 'Concord'}
capitals

{'MA': 'Boston', 'NH': 'Concord'}

You can access individual elements by key

In [144]:
capitals['MA']

'Boston'

It's an error to access a key that isn't there.

In [145]:
capitals['ME']

KeyError: 'ME'

But you can use the _get()_ function to safely return a default value

In [146]:
capitals.get('ME', 'not available')

'not available'

You can iterate over a dictionary with _for_ using the _items()_ function

In [147]:
capitals.items()

dict_items([('MA', 'Boston'), ('NH', 'Concord')])

In [148]:
for k, v in capitals.items():
    print('the capital of %s is %s' % (k, v))

the capital of MA is Boston
the capital of NH is Concord


in addition to items function dictionaries have keys() and values() which return lists of the keys or values

In [149]:
capitals.keys()

dict_keys(['MA', 'NH'])

In [150]:
list(capitals.keys())

['MA', 'NH']

In [151]:
capitals.values()

dict_values(['Boston', 'Concord'])

You can use the zip function to turn two lists into a dictionary 

In [152]:
states = ['MA', 'NH', 'ME']
cities = ['Boston', 'Concord', 'Agusta']
capitals = zip(states, cities)
capitals

Note that this is a zip object to complete turning it into a dictionary use the dict() funciton

In [153]:
dict(capitals)

{'MA': 'Boston', 'NH': 'Concord', 'ME': 'Agusta'}

## Exercise 14

#### 14.1 rewrite your code from 13.5 use a set rather than a list

#### 14.2 create a dictionary to map the organism to common names we used in get_common_name()

#### 14.3 loop through the dictionary and print out the keys and values

#### 14.4 use the __dict__.get() function to look for 'Mus musculus' and make sure to add a default (the second parameter)

#### 14.5 rewrite the fetching of our data rows from annotations.1.txt file (13.3) to make each row a dictionary using the zip function, use the header as the keys (you will have to split the header into a list)

#### 14.6 optional if you have time - rather than using zip iterate through the header and the line elements at one time using the enumerate function

#  15. Sorting lists

#### Python sorts lists by 'natural' order, either in place...

In [161]:
letters = ['a','x','t']
letters.sort()
letters

['a', 't', 'x']

In [162]:
numbers  = [1, 5, 20, 1.5]
numbers.sort()
numbers

[1, 1.5, 5, 20]

In [163]:
numberchars = ['1', '2', '100', '150']
numberchars.sort()
numberchars

['1', '100', '150', '2']

#### ... or as new list

In [164]:
numbers = [1,5,3,8]
sortednumbers = sorted(numbers)
numbers

[1, 5, 3, 8]

In [165]:
sortednumbers

[1, 3, 5, 8]

#### Reversing the direction is easy

In [166]:
sortednumbers.sort(reverse=True)
sortednumbers

[8, 5, 3, 1]

#### A key function provides flexibility in sorting

In [167]:
def case_insensitive(item):
    return item.lower()

words = ['and', 'or', 'But']
sortedwords = sorted(words)
sortedwords

['But', 'and', 'or']

In [168]:
sortedwords = sorted(words, key=case_insensitive)
sortedwords

['and', 'But', 'or']

## Exercise 15 

#### 15.1 we are going to sort the labeled_data by seqence length, to do this first write a key function which returns the sequencing length of a row as an integer 

In [ ]:
def seq_length(item):
   # fill in the body of this function (it should not be long)

#### 15.2 use that function to sort the labeled data with the longest sequence first

#### 15.2 let's create a dictionary of dictionaries here, create an empty dictionary called labeled_data_dictionary loop through sorted_labeled_data list and use Acccession as the key add the record dictionary as the value for a new record in labeled_data_dict.

# 16. Writing to file

using the open function to create a filehandle we can not only read as we've already seen with annotations.1.txt but write.  To open the file for writing use a 'w' for the second param for open.

In [172]:
test_file = 'data/test.txt'
fileh = open(test_file, 'w')
fileh.write('this is a line')
fileh.write('another line')
fileh.close()

to get a newline we have to add the newline character \n

In [173]:
test_file = 'data/test.txt'
fileh = open(test_file, 'w')
fileh.write('this is a line\n')
fileh.write('another line')
fileh.close()

opening with a, opens in append mode

In [174]:
test_file = 'data/test.txt'
fileh = open(test_file, 'a')
fileh.write('this is a line\n')
fileh.write('another line')
fileh.close()

## Exercise 16. write an annotated sample file

FASTA records have two parts, a description line, starting with '>', and the sequence, e.g.

    >NC_000012.12 Homo sapiens chromosome 12, GRCh38.p13 Primary Assembly     <-- Description line
    ATCGAGACCATCCTGGCCAACATAGTGAAAACCTTTCTCTACTAAAAATACAAAAATTAGCCAGGTATGG    <-- Sequence (DNA in this case)
    TCGAGAGGCTGAGGCAGGAGGATCGCTTAAACCTGGGAGGTAGAGGTTCCAGTGAGCTGAGATTGCGACA
    ...
    >NC_000013.12 Homo sapiens chromosome 13, GRCh38.p13 Primary Assembly

In this example, the first line is the description line, starting with a '>' and the second line starts the DNA sequence.
There can be multiple lines of sequence separated by newlines or just a single line.

The description line has further structure in that the characters between the '>' and the first whitespace are 
treated as the sequence record identifier, in this case NC_000012.12 or NC_000013.12

More than one FASTA record may be in a FASTA file.


First, open the samples.fa sequence file from data/chr12

#### 16.1 open data/chr12/samples.fa file for reading and loop through all the lines printing out only the sample descriptions which start with > 

#### 16.2 now let's create a dictionary called fastq_records then as you loop through the file if it's a sample name then set a variable called description to the sample name and create a key in fasta_records for the description set to an empty list, if the line is sequence then append it to that list

#### 16.3 create a new dictionary called annotated_description then loop through labeled_data_dict from 15.2 and add a record to annotated_description whith the same key (exp: 'sample1') and a new description with this format '>{accession} {organism} {gene_name}, {seq_type}'

#### 16.4 loop through fasta_records and print out only the sample name from each key excluding the < (one way to do this is to use string indexing)

#### 16.5 create a new dictionary called annotated_records, loop through the fasta_records and use the new annotated_description as the key and all the seq lines as the value (hint: you can use string slicing to remove the >, see section 9)

#### 16.6 write a file 'data/chr12/annotated-samples.fa' which contains the new description as a line then all the sequence lines, you can do this by looping through your annotated_records dictionary. Don't forget to add newlines.

# 17. Running shell commands from python 

### Before we get started let's open a bash terminal and try these commands in the unix/ bash environment outside of python 

_os.system_ runs a command in a bash shell and outputs stderr and stdout to the console.  It returns the shell return code (e.g. zero for success)

Because it goes to the console, your Python code does not capture the output.

Execution is synchronous, so your program has to wait until it's done.

Bash shell (or whatever your current shell is) interpolation is done so PATH is honored, redirection works, etc.

In [182]:
cmd = 'cat nonexistant_file.txt'

In [183]:
os.system(cmd)

256

You can check the return code for non-zero-ness

In [184]:
if os.system(cmd) != 0:
    print('Fail!')
else:
    print('Success!')

Fail!


In [185]:
cmd

'cat nonexistant_file.txt'

But you need to capture stderr to find out what happened

In [186]:
cmd = 'cat nonexistant_file.txt 2> stderr 1> stdout'

In [187]:
if os.system(cmd) != 0:
    stderrh = open('stderr', 'r')
    print(stderrh.read())

cat: nonexistant_file.txt: No such file or directory



### The subprocess _Popen()_ constructor allows more flexibility and power in the execution of shell commands.

The _Popen()_ constructor creates a process handle that can be used to capture stderr, stdout or pipe data into
stdin.

Run a process using Popen just like _os.system()_

In [188]:
import subprocess

In [189]:
cmd = 'echo "hello shell"'

In [190]:
proc = subprocess.Popen(cmd, shell=True)
proc.wait()

0

To capture stderr and stdout, use _PIPE_ and _.communicate()_

In [191]:
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [192]:
stdout, stderr = proc.communicate()
if proc.returncode == 0:
    print(stdout)
else:
    print('Fail %s' % stderr)

b'hello shell\n'


In Python 3, shell output is returned as a bytearray that must be decoded

In [193]:
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = proc.communicate()
if proc.returncode == 0:
    print(stdout.decode('ascii'))
else:
    print('Fail %s' % stderr)

hello shell



## Exercise 17

minimap2 is a command line tool for mapping query sequences to a reference.  This is useful for characterizing 
query sequences, SNP detection, finding orthologs (from close relatives), etc.  Command line usage is described 
as follows:

    Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]

where 'target' is the reference sequence (chr12.fa.gz for us)

use your annotated_sample_file from 16 as the query

In [261]:
target_file = 'data/chr12/chr12.fa.gz'

#### Let's try minimap2 in the terminal

We can type this command: ./minimap2 data/chr12/chr12.fa.gz data/chr12/annotated-samples.fa

./ is a special syntax that says to execute a script called minimap2, we can see the minimap2 script in our current python workshop directory

#### 17.1 set the cmd variable to a minimap2 command, note that it should start with ./minimap2 because this is a bash script that we want to execute.  Optionally, use string formatting like in section 8.

#### 17.2 use subprocess.popen to run the cmd, make sure to capture stdout, please be patient it may take a minute to run

#### 17.3 let's run the same command similar to 17.2 but let's create a dictionary with the following in it: returncode, stdout, stderr

#### 17.4 let's write a function called runcmd, the definition is there for you, fill in the body with the subprocess.Popen similar to 17.3 and return the dictionary. Create a variable called result and set it equal to the runcmd with the minimap2 cmd then pring out the returncode and output from the result.  

In [199]:
def runcmd(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE):
    
    

In [200]:
# you can test your function with this
result = runcmd(cmd)

# 18. parallel execution with a multiprocessing pool

Python cannot do real, parallel multithreading due to the [GIL](https://realpython.com/python-gil/).  The multiprocessing module simulates a threading library, but uses forked processes.

#### A multiprocessing Pool allows you to manage parallel processes easily

A multiprocessing Pool is an object that allows you to launch, manage, and retrieve results from a set of forked processes.

#### The _map_ function applies a set of values to a single argument function.  This is a useful way to do a "parameter sweep" type of execution.

```python
from multiprocessing import Pool
import os

def echo(echoable):
    os.system('echo %s && sleep 10' % echoable)
    
echoables = [
    'ajk',
    '123',
    'qwerty',
    'uiop',
    'lkjdsa',
]

numprocs = 3
pool = Pool(numprocs)
result = pool.map(echo,echoables)
```

_123_ <br/>
_ajk_ <br/>
_qwerty_ <br/>
_lkjdsa_ <br/>
_uiop_ <br/>


Let's try something in serial then commpare with the parallel

In [202]:
def greet(name, message):
    os.system('echo "Hi %s, %s" && sleep 5' % (name,message))
    return '%s was greeted' % name

greetings = [
    ('Howa', "What's up?"),
    ('Sidney', "How are you?"),
    ('Maria', "What're you thinking?"),
    ('folks', 'Sup!'),
]
import time
starttime = time.time()
for greeting in greetings:
    print(greet(greeting[0], greeting[1]))
elapsed = time.time() - starttime
print('%d seconds elapsed' % elapsed)

Howa was greeted
Sidney was greeted
Maria was greeted
folks was greeted
20 seconds elapsed


#### The _apply_async_ function allows you to apply many arguments and returns a 'handle' for interacting with the process.

In order for this to work in parallel, you'll need to collect the result handles in a list

note that apply_async takes a function and then a list of function arguments

In [203]:
from multiprocessing import Pool
import os
numprocs = 3
pool = Pool(numprocs)
results = []

starttime = time.time()
for greeting in greetings:
    result = pool.apply_async(greet, greeting)
    results.append(result)

for result in results:
    print(result.get())
elapsed = time.time() - starttime
print('%d seconds elapsed' % elapsed)

Howa was greeted
Sidney was greeted
Maria was greeted
folks was greeted
10 seconds elapsed


# Exercise 18. Run several minimap2 processes in parallel

#### 18.1 Create a function that runs minimap2 and return the result, use the run_cmd function you wrote

#### 18.2 use the following function arguments and set up a multiprocessing pool with 2 processes then add these queries to the pool with apply_async, don't forget to create a list from [target, query] when you pass it to apply_async because the function arguments must be a list

In [265]:
queries = [
    'data/chr12/annotated-samples.fa',
    'data/chr12/mouse.fa',
    'data/chr12/zebrafish.fa',
]
target = 'data/chr12/chr12.fa.gz'

# 19. Regular Expressions 

Python has a full-featured, Perl-ish regular expression syntax provided by the _re_ module

First, a simple search for DNA-ness in each of the fasta record sequences.

Using _re.search_ looks for at least one instance of the pattern

In [207]:
import re

In [247]:
seq = 'ATGGGGCGCAAATTGGACCCTACAAAGAAGGAGAAGCGGGGGCCAGGCCGAAAGGCCCGGAAGCAGAAGGGTGCCGAGACAGAACTCGCCAGATTCTTGCCTGCAGTAAGTGACGAAAATTCCAAGAGGCTGTCTAGTCGTGCTCGAAAGAGGGCAGCCAAGAGGAGGCTGGGTTCTGCTGAAGTCCCTAAGACAAATAAGTCCCCTGAGGCCAAACCATTGCCTGGAAAGCTACCAAAAGGAGCTGTCCAGACAGCTGGTAAGAAGGGACCCCAGTCCCTATTTAATGCTGCTCAAGGCAAGAAGCGCCCAGCACCTAGCAGTGATGAGGAAGAGGAGGAGGAAGACTCTGAAGAAGATGATGTGGTGAACCAGGGGGACCTCTGGGGCTCCGAGGATGATGCTGATATGGTAGATGACTATGGAGCTGACTCCAACTCTGAGGATGAGGAGGAAGGTGAAGAGCTGCTGCCCATTGAAAGAGCTGCTCGGAAGCAGAAGGTCCGGGAAGCTGCTGCTGGGGTCCAGTGGAGTGAAGAGGAGACGGAGGATGAGGAGGAAGAAGTGACCCCTGAGTCCGGCCCCTCAAAGGAGGAGGAGGCAGATGGGGGCCTGCAGATCAATGTGGATGAGGAACCATTTGTGCTGCCCCCTGCCGGGGAGATGGAGCAGGATGCCCAGGCTCCAGACCTGCAACGAGTTCACAAGCGGATCCAGGATATCGTGGGAATTCTGCGTGATTTTGGGGCTCAGCGGGAGGAAGGGCGGTCTCGTTCTGAATACCTGAACCGGCTCAAGAAGGATCTGGCCACTTACTACTCCTATGGAGACTTCCTGCTTGGCAAGCTCATGGACCTCTTCCCTCTGTCTGAGCTGGTGGAGTTCTTAGAAGCTAATGAGGTGCCTCGGCCCGTCACCCTCCGGACCAATACCTTGAAAACCCGACGCCGAGACCTTGCCCAGGCTCTAATCAATCGTGGGGTTAATCTGGATCCCCTGGGCAAGTGGTCAAAGACTGGACTAGTGGTATATGATTCTTCTGTGCCCATTGGTGCCACCCCCGAGTACCTGGCTGGGCACTACATGCTGCAGGGAGCCTCCAGCATGCTGCCCGTCATGGCCTTGGCACCCCAGGAACATGAGCGGATCCTGGACATGTGTTGTGCCCCTGGAGGAAAGACCAGCTACATGGCCCAGTTGATGAAGAATACGGGTGTGATCCTTGCCAATGACGCCAATGCTGAGCGGCTTAAGAGTGTTGTGGGCAACTTGCACCGGCTGGGAGTCACCAACACCGTTATCAGTCACTATGATGGGCGCCAGTTCCCCAAGGTGGTGGGGGGCTTTGACCGAGTGCTGCTGGATGCTCCCTGCAGTGGCACTGGGGTCATCTCCAAGGACCCAACCGTGAAGACTAACAAGGATGAGAAGGACATCCTGCGCTGTGCTCACCTCCAGAAGGAGTTGCTTCTGAGTGCTATTGACTCTGTGAATGCCACCTCCAAGACAGGAGGCTACCTTGTTTACTGCACCTGTTCTATCATGGTGGAAGAGAATGAGTGGGTGGTAGACTATGCTCTGAAAAAGAGGAATGTGCGACTGGTGCCCACAGGCCTAGACTTTGGCCAGGAGGGTTTTACCCGCTTTCGAGAAAGGCGCTTCCACCCCAGTCTGCGTTCTGCCCGACGCTTCTACCCTCATACCCACAATATGGATGGGTTCTTCATTGCCAAGTTCAAGAAATTCTCCAATTCTATCCCTCAGTCCGAGACAGGAAATTCTGAAACAGCCACACCTAAAAATGTAGACTTGCCTCAGGTCATCCCCAAGTCTGAGAACAGCAGCCAGCCAGCCAAAAAGGCCAAGGGGGCTGCAAAGACAAAGCAGCAGCTGCAGAAACAGCAACGTCCCAAGAAGGCCTCCTTCCAGAAGCCGAATGGCACCTCTAAAGGGGCAGACTCAGAATTGTCCACTGTACCTTCTGTCAGAAAGACCCAAGCTTCCTCCAGGTGCCAGGATAGCAGTCAGCCAGCTGGAAAAGCCGAAGGGATCAGGGAGTCCAAGGTGACTGGGAAGCTAAAGCAACGATCGCCTAAATTACAGTCCTCCAAGAAAGTTGCCTTCCTCAAGCAGAATGCCCCTCCCAAGGGCACAGACACAGAAACACCGGCTGTGTTATCCCCATCCAAGACTCAGGCCACCCTGAAACCTAAGGACCATCATCACCCCCTTGGAAGGGCCAAGGGGGTTGAGAAGCAGCAGTTGCCAGAGCAGCCTTTCAAGAAAGCTGCCTTCCAGAAACAGAATGACACCCCCAAGGGGCCTCAGCCTCCGACTGTGTCTCCCATCAGTTCCAGCCGCCCCCCACCAGCAAAGAGGAAGAAATCTCAGTCCAGGGGCAATGGCCAGCTGCTCCTGTCTTAGATGGTTGAAAACTAGACTGGGGTGGCTCACTGCCATTGTCACCGGGTTGGAACTCTTGCCTCTATGAGGATGCCTTCTCCACTGTGCATACCCATGAAATTTAATATATATATATATATATTATATATATATATATTAATATTATATAT'

In [209]:
if re.search(r'A', seq):
    print('found A')

found A


In [210]:
if re.search(r'U', seq):
    print('found U')
else:
    print('did not find U')

did not find U


You can search for multiple character patterns, like 'A' followed by 'T'

In [211]:
if re.search(r'AT', seq):
    print('found AT')

found AT


There are special quantifiers '+' (one or more) and '*' (zero or more)

In [212]:
if re.search(r'AT+', seq):
    print('found A followed by one or more T')

found A followed by one or more T


the . can be any character

In [213]:
if re.search(r'ATG.+', seq):
    print('found ATG, start codon followed by one or more char')

found ATG, start codon followed by one or more char


You can also search for character sets, e.g. one of A,T,C, or G, using square brackets [].

In [214]:
if re.search(r'[ATCG]', seq):
    print('found A or T or C or G')

found A or T or C or G


use | as or to look for one ore another group of characters

In [215]:
if re.search(r'TAG|TAA|TGA', seq):
    print('found a stop codon')

found a stop codon


In [216]:
match = re.search(r'(ATG.+TAG|TAA|TGA)', seq)
if match:
    print('Found a transcript looking thing %s' % match.group(1))

Found a transcript looking thing ATGGGGCGCAAATTGGACCCTACAAAGAAGGAGAAGCGGGGGCCAGGCCGAAAGGCCCGGAAGCAGAAGGGTGCCGAGACAGAACTCGCCAGATTCTTGCCTGCAGTAAGTGACGAAAATTCCAAGAGGCTGTCTAGTCGTGCTCGAAAGAGGGCAGCCAAGAGGAGGCTGGGTTCTGCTGAAGTCCCTAAGACAAATAAGTCCCCTGAGGCCAAACCATTGCCTGGAAAGCTACCAAAAGGAGCTGTCCAGACAGCTGGTAAGAAGGGACCCCAGTCCCTATTTAATGCTGCTCAAGGCAAGAAGCGCCCAGCACCTAGCAGTGATGAGGAAGAGGAGGAGGAAGACTCTGAAGAAGATGATGTGGTGAACCAGGGGGACCTCTGGGGCTCCGAGGATGATGCTGATATGGTAGATGACTATGGAGCTGACTCCAACTCTGAGGATGAGGAGGAAGGTGAAGAGCTGCTGCCCATTGAAAGAGCTGCTCGGAAGCAGAAGGTCCGGGAAGCTGCTGCTGGGGTCCAGTGGAGTGAAGAGGAGACGGAGGATGAGGAGGAAGAAGTGACCCCTGAGTCCGGCCCCTCAAAGGAGGAGGAGGCAGATGGGGGCCTGCAGATCAATGTGGATGAGGAACCATTTGTGCTGCCCCCTGCCGGGGAGATGGAGCAGGATGCCCAGGCTCCAGACCTGCAACGAGTTCACAAGCGGATCCAGGATATCGTGGGAATTCTGCGTGATTTTGGGGCTCAGCGGGAGGAAGGGCGGTCTCGTTCTGAATACCTGAACCGGCTCAAGAAGGATCTGGCCACTTACTACTCCTATGGAGACTTCCTGCTTGGCAAGCTCATGGACCTCTTCCCTCTGTCTGAGCTGGTGGAGTTCTTAGAAGCTAATGAGGTGCCTCGGCCCGTCACCCTCCGGACCAATACCTTGAAAACCCGACGCCGAGACCTTGCCCAGGCTC

The _split()_ function allows you to break a string based on a regular expression.

Find potential genes by splitting chr12 on stop codons followed by a lot of T

In [272]:
os.system('gzip -k -d data/chr12/chr12.fa.gz')
chr12 = []
with open('data/chr12/chr12.fa', 'r') as fileh:
    for line in fileh:
        if not line.startswith('>'):
            chr12.append(line.strip())

In [269]:
len(chr12)

1903934

In [244]:
chr12 = ''.join(chr12)

In [245]:
coding = re.split(r'(T[GA][GA]T{20,})', chr12)

In [246]:
len(coding)

885

## Exercise 19

#### 19.1 loop through fasta_records and print out the key for any record that contains a start codon, since the dictionary value of fasta_records is a list, join it to a string using empty string as the joiner.

#### 19.2 loop through fasta_records and print out any potential transcripts

# 20. Final Exercise: Perform a virtual restriction fragment digest on chromosome 12 and map the annotated samples to the largest fragments

* Open the file 'data/chr12/chr12.fa' and read the chromosome 12 sequence into a list of lines
* Cut it with the BisI restriction enzyme (recognizes GCAGC, GCTGC, GCGGC and GCCGC) (use re.split)
* Sort the unique fragments by size (use a custom sort function on length)
* Write the 3 largest fragments to 3 separate FASTA files
* Run minimap2 with the annotated samples against each of the 3 largest fragments